In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import pandas as pd
# Load data
NSDUH_2020 = pd.read_csv('NSDUH_2020_Tab.txt', sep = '\t', low_memory=False)

In [4]:
NSDUH_2020.head()

,QUESTID2,FILEDATE,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,...,GOVTPROG,INCOME,POVERTY3,PDEN10,COUTYP4,MAIIN102,AIIND102,ANALWTQ1Q4_C,VESTRQ1Q4_C,VEREP
0,10000182,10/25/2021,1,99,99,16,9999,99,1,30,...,2,4,3.0,3,3,2,2,16186.579071,40048,2
1,10001831,10/25/2021,1,99,99,8,9999,99,1,24,...,2,3,3.0,2,2,2,2,16809.072854,40022,2
2,10002722,10/25/2021,2,99,99,991,9991,91,91,91,...,2,2,3.0,1,1,2,2,1379.866862,40011,1
3,10002897,10/25/2021,2,99,99,991,9991,91,91,91,...,2,4,3.0,1,1,2,2,422.776340,40007,1
4,10004677,10/25/2021,2,4,4,991,9991,91,91,91,...,2,4,3.0,2,2,2,2,609.829709,40031,1


In [5]:
NSDUH_2020.shape

(32893, 2890)

In [6]:
dfo = NSDUH_2020[~NSDUH_2020['SCHFELT'].isna()]
dfo.shape

(5500, 2890)

In [7]:
# make a vector of substance use column names
substance_cols = [# QUANTITATIVE VALUES FOR FREQUENCY OF USE
                       'IRALCFY', # ALCOHOL FREQUENCY PAST YEAR (1-365)
                       'IRMJFY', # MARIJUANA FREQUENCY PAST YEAR (1-365)
                       'IRCIGFM', #CIGARETTE FREQUENCY PAST MONTH (1-30)
                       'IRSMKLSS30N', # SMOKELESS TOBACCO FREQUENCY PAST MONTH (1-30)
                       'IRALCFM', # ALCOHOL FREQUENCY PAST MONTH (1-30)
                       'IRMJFM', # MARIJUANA FREQUENCY PAST MONTH (1-30)
                       
                       # QUANTITATIVE VALUES FOR AGE OF FIRST USE
                       'IRCIGAGE', # CIGARETTE AGE OF FIRST USE (1-55), 991=NEVER USED
                       'IRSMKLSSTRY', # SMOKELESS TOBACCO AGE OF FIRST USE (1-70), 991=NEVER USED
                       'IRALCAGE', # ALCOHOL AGE OF FIRST USE (1-66), 991=NEVER USED
                       'IRMJAGE', # MARIJUANA AGE OF FIRST USE (1-83), 991=NEVER USED
                       
                       # BINARY CATEGORIES FOR USE AT ALL
                       'MRJFLAG', # MARIJUANA EVER USED (0=NEVER, 1=EVER)
                       'ALCFLAG', # ALCOHOL EVER USED (0=NEVER, 1=EVER)
                       'TOBFLAG', # ANY TOBACCO EVER USED (0=NEVER, 1=EVER)
                       
                       # MULTICLASS CATEGORIES FOR FREQUENCY OF USE 
                       'ALCYDAYS', # NUMBER OF DAYS OF ALCOHOL IN PAST YEAR (1-5 CATEGORIES, 6=NONE)
                       'MRJYDAYS', # NUMBER OF DAYS OF MARIJUANA IN PAST YEAR (1-5 CATEGORIES, 6=NONE)
                       'ALCMDAYS', # NUMBER OF DAYS OF ALCOHOL IN PAST MONTH (1-4 CATEGORIES, 5=NONE)
                       'MRJMDAYS', # NUMBER OF DAYS OF MARIJUANA IN PAST MONTH (1-4 CATEGORIES, 5=NONE)
                       'CIGMDAYS', # NUMBER OF DAYS OF CIGARETTES IN PAST MONTH (1-5 CATEGORIES, 6=NONE)
                       'SMKLSMDAYS' # NUMBER OF DAYS OF SMOKELESS TOBACCO IN PAST MONTH (1-4 CATEGORIES, 5=NONE)
]

# make a vector of demographic column names
demographic_cols = ['IRSEX', # BINARY SEX (1=MALE, 2=FEMALE)
                  'NEWRACE2', # RACE (7 CATEGORIES)
                  'HEALTH2', # OVERALL HEALTH (4 CATEGORIES)
                  'EDUSCHLGO', # NOW GOING TO SCHOOL (1=YES, 2=NO)
                  'EDUSCHGRD2', # WHAT GRADE IN NOW/WILL BE IN (11 CATEGORIES, 98,99= BLANK/SKIP)
                  'EDUSKPCOM', #HOW MANY DAYS SKIPPED SCHOOL IN PAST MONTH (1-30, 94/97/98/99=BLANK/SKIP)
                  'IMOTHER', # FOR YOUTH, MOTHER IN HOUSEHOLD (1=YES, 2=NO, 3=DON'T KNOW, 4=OVER 18)
                  'IFATHER', # FOR YOUTH, FATHER IN HOUSEHOLD (1=YES, 2=NO, 3=DON'T KNOW, 4=OVER 18)
                  'INCOME', # TOTAL FAMILY INCOME (4 CATEGORIES)
                  'GOVTPROG', # GOT GOV ASSISTANCE (1=YES, 2=NO)
                  'POVERTY3', # POVERTY LEVEL (4 CATEGORIES)
                  'PDEN10', # POPULATION DENSITY (1= >1M PEOPLE, 2=<1M PEOPLE, 3=CAN'T BE DETERMINED)
                  'COUTYP4' # METRO SIZE STATUS (1=LARGE METRO, 2=SMALL METRO, 3=NONMETRO)
]

# select columns of interest
df_youth = dfo.loc[:, 'SCHFELT':'RLGFRND']
df_substance = dfo.loc[:, substance_cols]
df_demog = dfo.loc[:, demographic_cols]

# combine into one data frame
df = pd.concat([df_substance, df_youth, df_demog], axis=1)

# Fix metadata

# make vector of columns from the data that should be converted to factors, unordered and ordered
unordered_factor_cols = list(df_youth.columns) + ['MRJFLAG','ALCFLAG','TOBFLAG','IRSEX','NEWRACE2',
                                                  'EDUSCHLGO','IMOTHER','IFATHER','GOVTPROG','PDEN10','COUTYP4']
ordered_factor_cols = ['EDUSCHGRD2','HEALTH2','POVERTY3','INCOME']

# convert to factors
df[unordered_factor_cols] = df[unordered_factor_cols].astype('category')
df[ordered_factor_cols] = df[ordered_factor_cols].apply(lambda x: pd.Categorical(x, ordered=True))

# Save data to CSV file without any encoding
df.to_csv('youth_data.csv', index=False, encoding='utf-8-sig')